In [1]:
import os

In [ ]:
import os
print(os.getcwd())

In [3]:
os.chdir("../")

In [ ]:
import os
print(os.getcwd())

In [ ]:
import dagshub
dagshub.init(repo_owner='KartikGarg20526', repo_name='End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC', mlflow=True)

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [16]:
import pandas as pd
from box import ConfigBox
from src.mlops_project.logger import logging
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = ConfigBox(read_yaml(config_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/KartikGarg20526/End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC.mlflow",  
        )

        return model_evaluation_config

In [18]:
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from urllib.parse import urlparse
import mlflow.sklearn
import joblib

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)

        return accuracy, precision, recall, f1
    
    def log_into_mlflow(self):
        try:
            logging.info("Reading test data")
            test_data = pd.read_csv(self.config.test_data_path)

            logging.info("Loading model")
            model = joblib.load(self.config.model_path)

            test_x = test_data.drop([self.config.target_column], axis=1)
            test_y = test_data[[self.config.target_column]]

            logging.info("Logging into mlflow")
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                
                logging.info("Calculating metrics")
                predictions = model.predict(test_x)

                accuracy, precision, recall, f1 = self.eval_metrics(test_y, predictions)

                scores = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

                logging.info("Saving metrics in json file")
                save_json(path=Path(self.config.metric_file_name), data=scores)

                logging.info("Logging params into mlflow")
                mlflow.log_params(self.config.all_params)

                logging.info("Logging metrics into mlflow")
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1-score", f1)

                # Model registry does not work with file store
                logging.info("Logging model into mlflow")
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
                else:
                    mlflow.sklearn.log_model(model, "model")

        except Exception as e:
            raise e

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e